In [107]:
import sys
import os
import csv
import math
import pandas as pd
import numpy as np
import datetime

import math
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasClassifier
# from keras.utils import np_utils
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, KFold, train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
import os

from sklearn.linear_model import LogisticRegressionCV, LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB, ComplementNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, OneClassSVM
from sklearn.neural_network import BernoulliRBM, MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn import preprocessing

import xgboost as xgb

import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats

# Creating Separate Datasets for Suite and App

In [61]:
#If you already have the files:
df_SL = pd.read_csv('EUModel_SuiteDataMerged.csv')
df_AL = pd.read_csv('EUModel_AppDataMerged.csv')
#If you don't, here's how it's created:

C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (57,59,69,70,73,74,75,76,78,79,80,81,83,98,99,130,153,177,179,181,182,183,184,189,190,191,194,196,198,203,208,209,210,211,212,215,217,218,230,231,232,233,234,238,358) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Incorporating the Performance Correlation Load Time Data:

In [3]:
df_L = pd.read_csv('C:/Users/v-Jaand/Downloads/PerformanceCorrelation/WEP_Boot_Launch_nps_2019_02_01_to_2019_08_31.tsv', sep='\t')

In [4]:
print(df_L.shape)
df_L.loc[:,['Feedback_ID','Device_ProcessSessionId','SessionId']].head()

(184397, 32)


,Feedback_ID,Device_ProcessSessionId,SessionId
0,0038e13a-c5d5-4183-b4b5-672cc3377e72,0038e13a-c5d5-4183-b4b5-672cc3377e72,0038e13a-c5d5-4183-b4b5-672cc3377e72
1,00be97ea-8812-415c-9815-b9ded017d2ab,00be97ea-8812-415c-9815-b9ded017d2ab,00be97ea-8812-415c-9815-b9ded017d2ab
2,00d5c824-b48e-4b13-9686-8f4323f34453,00d5c824-b48e-4b13-9686-8f4323f34453,00d5c824-b48e-4b13-9686-8f4323f34453
3,00dd8196-810d-409a-8545-af5165bdd715,00dd8196-810d-409a-8545-af5165bdd715,00dd8196-810d-409a-8545-af5165bdd715
4,00eb4d07-2db0-4ddf-ac0d-f049b5cdbf2a,00eb4d07-2db0-4ddf-ac0d-f049b5cdbf2a,00eb4d07-2db0-4ddf-ac0d-f049b5cdbf2a


In [5]:
df_L = df_L[~(df_L.SessionId!=df_L.Feedback_ID)]
df_L = df_L[~(df_L.Device_ProcessSessionId!=df_L.Feedback_ID)]
df_L = df_L[~(df_L.SessionId!=df_L.Device_ProcessSessionId)]
print(df_L.shape)

(184395, 32)


## O365 and App Files from DataMart/Kusto:

In [6]:
def transform_rating(rating):
    '''Input: Column of Data with NPS Field (on a scale of 1 to 5...)
    Output: Column with either Promoter/Detractor/Passive label, or the corresponding weights, based on datatype'''
    
    if rating == 5:
        return 'Promoter'
    elif rating <= 3:
        return 'Detractor'
    elif rating == 4:
        return 'Passive'
    else:
        return 'Invalid'
    

def NPS_rating(Rating_feel):
    if Rating_feel == 5.0:
        return 100
    elif Rating_feel== 4.0:
        return 0
    elif Rating_feel == 3.0:
        return -100
    elif Rating_feel == 2.0:
        return -100
    elif Rating_feel == 1.0:
        return -100
    else:
        return  0

def create_NPSdataframe(data_file, build_file):
    '''Inputs: Raw Dataset and corresponding OS Builds
       Outputs: Cleaned Dataset'''
    iw3= pd.read_csv(data_file,index_col= 0)
    Build= pd.read_csv(build_file)
    
    iw3.rename({'User_Country': 'User_UserCountry',
                'Tenant_Country':'Tenant_TenantCountry'}, 
               axis='columns', inplace=True)

    iw3.columns = [x.partition('_')[2] for x in iw3.columns.to_list()]
    iw3 = iw3[list(iw3.columns[~iw3.columns.duplicated()])]

    iw3['minorbuild'] = iw3['OfficeBuild'].str.split('.', expand=True)[2]
    Build['OB_ThirdPart'] = Build['OfficeBuildPrefix'].str.split('.', expand=True)[2]
    Build =Build.set_index('OB_ThirdPart')['ForkMth'].to_dict()
    #Build =Build.set_index('OB_ThirdPart')['OfficeForkBuild'].to_dict()
    iw3['fork'] = iw3['minorbuild'].astype(float).map(Build)
        
    print('Mapped Builds to Forks')

    #iw3.rename(index=str,columns={"Source": "Product"},inplace=True)

    #iw3#.loc[iw['TenantState'] !='Unknown'] #This is a filter we sometimes use, but not currently
    # iw3=iw3.loc[iw3['App'] != 'Other']
    # iw3=iw3.loc[iw3['App'] != 'OneNote']
    # iw3=iw3[iw3['Product'] !='Desktop Visio']
    # iw3=iw3[iw3['Product'] !='Desktop OneNote']
    
    #iw3["SystemLocaleTag"] = iw3["SystemLocaleTag"].astype(str)
    #iw3["Language"] = iw3["SystemLocaleTag"].apply(lambda x: x.split("-")[0])

    #iw3['fork2']=iw3['fork'].map(lambda x: fork2(x))

    iw3["DateTime"] = iw3["DateTime"].apply(lambda x: pd.to_datetime(x))
    #iw3= iw3.loc[iw3["LongDateTime"]>= datetime.strptime('2018-05-01' , '%Y-%m-%d').date()]

#     iw3['Feed_year'] = pd.DatetimeIndex(iw3['LongDateTime']).year
#     iw3['Feed_month'] = pd.DatetimeIndex(iw3['LongDateTime']).month
#     iw3['month_year'] = iw3.LongDateTime.dt.to_period('M')
#     iw3['week_year'] = iw3.LongDateTime.dt.to_period('W-SAT')
#     iw3['Feed_week'] = pd.DatetimeIndex(iw3['LongDateTime']).week
#     print('Analysis dates: {0} to {1}'.format(iw3["LongDateTime"].min().strftime('%Y-%m-%d'), iw3["LongDateTime"].max().strftime('%Y-%m-%d')))

    iw3['NPS'] = iw3['Rating'].apply(transform_rating)
    iw3['ratingvalue'] = iw3['Rating'].apply(NPS_rating)

    # iw3.loc[iw3['SurveyRatingQuestion'] == 'How likely are you to recommend Office 365 to a friend or colleague?', 'SurveyType'] = 'Suite'
    # iw3.loc[iw3['SurveyRatingQuestion'] == 'How likely are you to recommend our application to a friend or colleague?', 'SurveyType'] = 'App'    
    iw3.loc[iw3['SurveyName'].str.startswith('Suite'), 'SurveyType'] = 'Suite'
    iw3.loc[iw3['SurveyName'].str.startswith('App'), 'SurveyType'] = 'App'
    iw3['SurveyApp'] = iw3['SurveyType'] + iw3['App']
    
    #Redsigning Discretized Variables:
    iw3.loc[iw3['ScreenDpi']<120,'ScreenDPI'] = 'Less than 120'
    iw3.loc[iw3['ScreenDpi']>=120,'ScreenDPI'] = '120 or Greater'
    iw3['RamGB'] = iw3['RamMB']/1024.0
    iw3['ProcSpeedGHz'] = iw3['ProcSpeedMHz']//1000
    
#     count_list = ['TotalUsers'#,'ProPlusEnabledUsers'
#     ]
    
    iw3['SysVolUsagePercent'] = 1-iw3['SysVolFreeSpaceMB']/iw3['SysVolSizeMB']

#     for i, item in enumerate(count_list):
#         var_name = item+'_Ord'
#         iw3.loc[iw3[item]==1,var_name] = '1'
#         iw3.loc[iw3[item].between(2,9),var_name] = 'Small Group (2-9)'
#         iw3.loc[iw3[item].between(10,149),var_name] = 'Medium Group (10-149)'
#         iw3.loc[iw3[item]>=150,var_name] = 'Large Group (150+)'
    
    iw3['Resolution']=iw3['HorizontalResolution'].astype(str)+'x'+iw3['VerticalResolution'].astype(str)
    iw3[iw3.columns[list(iw3.dtypes=='category')]] = iw3[iw3.columns[list(iw3.dtypes=='category')]].astype(str)
    
    return iw3


In [7]:
df_S = create_NPSdataframe('EUModel_SuiteData.csv','../Build_to_fork_July2019.csv')

C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2901: DtypeWarning: Columns (57,59,69,70,73,74,75,76,78,79,80,81,83,98,99,130,153,177,179,181,182,183,184,189,190,191,194,196,198,203,208,209,210,211,212,215,217,218,230,231,232,233,234,238) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Mapped Builds to Forks


In [8]:
df_A = create_NPSdataframe('EUModel_AppData.csv','../Build_to_fork_July2019.csv')

C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2901: DtypeWarning: Columns (57,59,69,70,73,74,75,76,78,79,80,81,83,98,99,130,153,177,179,181,182,183,184,189,190,191,194,196,198,203,208,209,210,211,212,215,217,218,230,231,232,233,234,238) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Mapped Builds to Forks


In [9]:
df_L.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184395 entries, 0 to 184396
Data columns (total 32 columns):
Feedback_ID                 184395 non-null object
Device_ProcessSessionId     184395 non-null object
Date                        184395 non-null object
Feedback_SurveyName         184395 non-null object
LongDate                    184395 non-null object
Feedback_Platform           184395 non-null object
Feedback_App                184395 non-null object
Rating                      184395 non-null int64
Feedback_RatingValue        184395 non-null int64
Feedback_RatingScale        184395 non-null int64
Verbatim                    56374 non-null object
Feedback_TenantId           111260 non-null object
Device_SysVolSizeMB         182178 non-null float64
Device_SysVolFreeSpaceMB    182178 non-null float64
Feedback_UserType           184395 non-null object
rn                          184395 non-null int64
Time                        184395 non-null object
ClientId                 

In [10]:
df_S.info()

<class 'pandas.core.frame.DataFrame'>
Index: 83403 entries, a66f19ce-fc03-493e-bbc1-caa130fdbe75 to fee48244-6c54-4c97-ac01-277474fde90e
Columns: 329 entries, DateTime to Resolution
dtypes: datetime64[ns](1), float64(195), int64(6), object(127)
memory usage: 210.0+ MB


In [11]:
df_S.columns[df_S.columns.str.contains('Device')]

Index(['DeviceManufacturer', 'DeviceModel'], dtype='object')

In [12]:
cols = []
dupe_cols = []
for i in df_S.columns:
    if i in cols:
        print(i)
        dupe_cols.append(i)
    else:
        cols.append(i)

Source
ProgramId


In [13]:
df_S.loc[:,dupe_cols].head()

,Source,Source,ProgramId,ProgramId
Feedback_ID,,,,
a66f19ce-fc03-493e-bbc1-caa130fdbe75,Desktop Excel,NaN,NaN,NaN
8ac1dc00-8f9c-4fae-a74f-8dae2a06001a,Desktop Powerpoint,NaN,NaN,NaN
c0bd1047-b72d-4a46-a0b3-ac1fa79a955d,Desktop Outlook,NaN,NaN,NaN
390052ef-15f5-4433-8b30-f14a6db71154,Desktop Excel,NaN,NaN,NaN
636b57a0-cfa4-4b30-a40a-c2af227a268c,Desktop Powerpoint,NaN,NaN,NaN


In [14]:
df_S.loc[:,dupe_cols].drop(['ProgramId'],axis=1,inplace=True)
df_A.loc[:,dupe_cols].drop(['ProgramId'],axis=1,inplace=True)

In [15]:
df_S.loc[:,df_S.columns[df_S.columns.str.contains('Country')]].head()

,ClientCountry,UserCountry,TenantCountry,CountryCode,MSSalesCountryCode,MSSalesCountryName
Feedback_ID,,,,,,
a66f19ce-fc03-493e-bbc1-caa130fdbe75,PH,Philippines,Norway,NO,NaN,NaN
8ac1dc00-8f9c-4fae-a74f-8dae2a06001a,US,United States,NaN,NaN,NaN,NaN
c0bd1047-b72d-4a46-a0b3-ac1fa79a955d,US,United States,NaN,NaN,NaN,NaN
390052ef-15f5-4433-8b30-f14a6db71154,US,United States,United States,US,NaN,NaN
636b57a0-cfa4-4b30-a40a-c2af227a268c,US,United States,United States,US,NaN,NaN


## Merge on Device_ProcessSessionId

In [16]:
def clean_PC_data(df):
    df['DateTime'] = pd.to_datetime(df['Date'])
    week_cutoff = df.DateTime.max().to_period('D') -7
    biweek_cutoff = df.DateTime.max().to_period('D') -14
    month_cutoff = df.DateTime.max().to_period('D') -28
    df.sort_values(by='DateTime',inplace=True)
    value_key = df.groupby(['Device_ProcessSessionId'])['Feedback_RatingValue'].last().to_dict()
    sessions_7days = df[df.DateTime.dt.to_period('D')>week_cutoff].groupby(['DeviceId'])['SessionId'].count().to_dict()
    sessions_14days = df[df.DateTime.dt.to_period('D')>biweek_cutoff].groupby(['DeviceId'])['SessionId'].count().to_dict()
    sessions_28days = df[df.DateTime.dt.to_period('D')>month_cutoff].groupby(['DeviceId'])['SessionId'].count().to_dict()
    df['ratingvalue'] = df.Device_ProcessSessionId.map(value_key)
    df = df[df.ratingvalue.notnull()]
    df['Sessions_7'] = df.Device_ProcessSessionId.map(sessions_7days).fillna(0)
    df['Sessions_14'] = df.Device_ProcessSessionId.map(sessions_14days).fillna(0)
    df['Sessions_28'] = df.Device_ProcessSessionId.map(sessions_28days).fillna(0)
    df.drop_duplicates(subset=['Device_ProcessSessionId'],keep='last',inplace=True)
    df['SysVolUsagePercent'] = 1-df['Device_SysVolFreeSpaceMB']/df['Device_SysVolSizeMB']
    return df

df_L = clean_PC_data(df_L)

In [17]:
df_L.Device_ProcessSessionId.value_counts().head()

031452dd-fc64-4a7a-8ff4-59593d7e6b7f    1
65c5106b-c4d0-4d92-a87b-29506c2a1dc1    1
6b607e82-5674-472b-a2bf-8b57619bd327    1
bafec25c-7062-4a22-8be2-b849c560ef36    1
8cc2b043-90cf-44e8-9fcc-2b3e03c0f621    1
Name: Device_ProcessSessionId, dtype: int64

In [18]:
df_S.sort_values(by='DateTime',inplace=True)
value_key = df_S.groupby(['ProcessSessionId'])['RatingValue'].last().to_dict()
df_S['ratingvalue'] = df_S.ProcessSessionId.map(value_key)
df_S = df_S[df_S.ratingvalue.notnull()]
df_S.drop_duplicates(subset=['ProcessSessionId'],keep='last',inplace=True)

In [19]:
df_A.sort_values(by='DateTime',inplace=True)
value_key = df_A.groupby(['ProcessSessionId'])['RatingValue'].last().to_dict()
df_A['ratingvalue'] = df_A.ProcessSessionId.map(value_key)
df__A = df_A[df_A.ratingvalue.notnull()]
df_A.drop_duplicates(subset=['ProcessSessionId'],keep='last',inplace=True)

In [20]:
df_A.head()

,DateTime,Source,Source,Platform,UserType,TenantId,Rating,Verbatim,SurveyName,RatingValue,...,fork,NPS,ratingvalue,SurveyType,SurveyApp,ScreenDPI,RamGB,ProcSpeedGHz,SysVolUsagePercent,Resolution
Feedback_ID,,,,,,,,,,,,,,,,,,,,,
345f0eee-c1a4-4170-8f97-070700f2babc,2019-02-01 00:05:13,Desktop Excel,NaN,Windows Desktop,Consumer,NaN,4,NaN,App EndUser NPS,0,...,NaN,Passive,0,App,AppExcel,NaN,8.0,3.0,0.156354,nanxnan
8a7765aa-9b04-4c9e-8926-fe085bd43bd0,2019-02-01 00:10:06,Desktop Word,NaN,Windows Desktop,Consumer,NaN,5,Simple,App EndUser NPS,100,...,NaN,Promoter,100,App,AppWord,NaN,2.0,2.0,0.539740,nanxnan
daea175e-d64b-46da-b114-7ca09fbbfb96,2019-02-01 00:15:05,Desktop Outlook,NaN,Windows Desktop,Commercial,4469a43f-7481-45e9-b332-74e9101b05c8,5,NaN,App EndUser NPS,100,...,NaN,Promoter,100,App,AppOutlook,NaN,16.0,2.0,0.112771,nanxnan
f529e94c-b4c0-47c3-aa34-5c707e7175c3,2019-02-01 00:30:01,Desktop Word,NaN,Windows Desktop,Commercial,0e5bf3cf-1ff4-46b7-9176-52c538c22a4d,5,NaN,App EndUser NPS,100,...,NaN,Promoter,100,App,AppWord,NaN,8.0,1.0,0.365616,nanxnan
54e8dff8-6598-4f20-820a-f89dd918dfef,2019-02-01 00:30:04,Desktop Powerpoint,NaN,Windows Desktop,Consumer,NaN,5,PPT is a very good sw to make a good presentat...,App EndUser NPS,100,...,NaN,Promoter,100,App,AppPowerPoint,NaN,2.0,2.0,0.303912,nanxnan


In [21]:
df_SL = df_S.set_index('ProcessSessionId').join(df_L.set_index('Device_ProcessSessionId'),
                                              lsuffix='_Kusto', rsuffix='_other')

In [22]:
df_AL = df_A.set_index('ProcessSessionId').join(df_L.set_index('Device_ProcessSessionId'),
                                              lsuffix='_Kusto', rsuffix='_other')

In [23]:
df_SL.to_csv('EUModel_SuiteDataMerged.csv')
df_AL.to_csv('EUModel_AppDataMerged.csv')

# Feature Engineering with Joined Dataset

In [62]:
print(df_SL.columns[df_SL.columns.str.contains('_Kusto')])
df_SL.columns[df_SL.columns.str.contains('_other')]

Index(['DateTime_Kusto', 'Rating_Kusto', 'Verbatim_Kusto',
       'ProcessorCount_Kusto', 'ProcSpeedMHz_Kusto', 'RamMB_Kusto',
       'ratingvalue_Kusto', 'SysVolUsagePercent_Kusto'],
      dtype='object')


Index(['Rating_other', 'Verbatim_other', 'ProcessorCount_other',
       'ProcSpeedMHz_other', 'RamMB_other', 'DateTime_other',
       'ratingvalue_other', 'SysVolUsagePercent_other'],
      dtype='object')

In [63]:
print(df_AL.columns[df_SL.columns.str.contains('_Kusto')])
df_AL.columns[df_SL.columns.str.contains('_other')]

Index(['DateTime_Kusto', 'Rating_Kusto', 'Verbatim_Kusto',
       'ProcessorCount_Kusto', 'ProcSpeedMHz_Kusto', 'RamMB_Kusto',
       'ratingvalue_Kusto', 'SysVolUsagePercent_Kusto'],
      dtype='object')


Index(['Rating_other', 'Verbatim_other', 'ProcessorCount_other',
       'ProcSpeedMHz_other', 'RamMB_other', 'DateTime_other',
       'ratingvalue_other', 'SysVolUsagePercent_other'],
      dtype='object')

In [64]:
common_fields = list(df_SL.columns[df_SL.columns.str.contains('_Kusto')])+list(df_SL.columns[df_SL.columns.str.contains('_other')])

In [65]:
print(df_SL.loc[:,common_fields].shape)
df_SL.loc[:,common_fields].head(10)

(83403, 16)


,DateTime_Kusto,Rating_Kusto,Verbatim_Kusto,ProcessorCount_Kusto,ProcSpeedMHz_Kusto,RamMB_Kusto,ratingvalue_Kusto,SysVolUsagePercent_Kusto,Rating_other,Verbatim_other,ProcessorCount_other,ProcSpeedMHz_other,RamMB_other,DateTime_other,ratingvalue_other,SysVolUsagePercent_other
0,2019-02-01 00:00:06,4,NaN,4.0,3193.0,4096.0,0,0.141071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-02-01 00:04:09,3,NaN,4.0,2594.0,8192.0,-100,0.168609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-02-01 00:05:02,4,NaN,4.0,3493.0,4096.0,0,0.089559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-02-01 00:05:06,5,NaN,8.0,2112.0,16384.0,100,0.108312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-02-01 00:10:04,4,NaN,8.0,2712.0,16384.0,0,0.344600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2019-02-01 00:14:06,5,Don't increase your annual price don't lower ...,4.0,2400.0,16384.0,100,0.261836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2019-02-01 00:14:06,5,NaN,8.0,1800.0,8192.0,100,0.565450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2019-02-01 00:14:06,5,NaN,8.0,2394.0,8192.0,100,0.061884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2019-02-01 00:20:06,4,NaN,4.0,2794.0,32768.0,0,0.410083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2019-02-01 00:34:06,4,NaN,4.0,2400.0,4096.0,0,0.152233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Some Telemetry Data does not line up between datasets:

In [66]:
test = df_SL[df_SL.RamMB_other.notnull()].RamMB_Kusto != df_SL[df_SL.RamMB_other.notnull()].RamMB_other

print(df_SL[df_SL.RamMB_other.notnull()].loc[test,common_fields].shape)
df_SL[df_SL.RamMB_other.notnull()].loc[test,common_fields].head()

(1316, 16)


,DateTime_Kusto,Rating_Kusto,Verbatim_Kusto,ProcessorCount_Kusto,ProcSpeedMHz_Kusto,RamMB_Kusto,ratingvalue_Kusto,SysVolUsagePercent_Kusto,Rating_other,Verbatim_other,ProcessorCount_other,ProcSpeedMHz_other,RamMB_other,DateTime_other,ratingvalue_other,SysVolUsagePercent_other
9629,2019-02-22 14:20:36,4,NaN,NaN,NaN,NaN,0,NaN,4.0,NaN,4.0,1992.0,4096.0,2019-02-22 06:20:36,0.0,NaN
11719,2019-02-28 04:30:18,3,It is slightly more difficult than Excel or Wo...,NaN,NaN,NaN,-100,NaN,3.0,It is slightly more difficult than Excel or Wo...,4.0,2400.0,8192.0,2019-02-27 20:30:18,-100.0,NaN
12634,2019-03-02 16:05:07,3,NaN,NaN,NaN,NaN,-100,NaN,3.0,NaN,8.0,1992.0,16384.0,2019-03-02 08:05:07,-100.0,NaN
12890,2019-03-03 15:45:09,5,NaN,NaN,NaN,NaN,100,NaN,5.0,NaN,4.0,2494.0,8192.0,2019-03-03 07:45:09,100.0,NaN
12892,2019-03-03 15:59:08,3,It would be interesting to integrate Office wi...,NaN,NaN,NaN,-100,NaN,3.0,It would be interesting to integrate Office wi...,4.0,2808.0,16384.0,2019-03-03 07:59:08,-100.0,NaN


In [67]:
test = df_SL[df_SL.ProcessorCount_other.notnull()].ProcessorCount_Kusto != df_SL[df_SL.ProcessorCount_other.notnull()].ProcessorCount_other

print(df_SL[df_SL.ProcessorCount_other.notnull()].loc[test,common_fields].shape)
df_SL[df_SL.ProcessorCount_other.notnull()].loc[test,common_fields].head()

(26, 16)


,DateTime_Kusto,Rating_Kusto,Verbatim_Kusto,ProcessorCount_Kusto,ProcSpeedMHz_Kusto,RamMB_Kusto,ratingvalue_Kusto,SysVolUsagePercent_Kusto,Rating_other,Verbatim_other,ProcessorCount_other,ProcSpeedMHz_other,RamMB_other,DateTime_other,ratingvalue_other,SysVolUsagePercent_other
9629,2019-02-22 14:20:36,4,NaN,NaN,NaN,NaN,0,NaN,4.0,NaN,4.0,1992.0,4096.0,2019-02-22 06:20:36,0.0,NaN
11719,2019-02-28 04:30:18,3,It is slightly more difficult than Excel or Wo...,NaN,NaN,NaN,-100,NaN,3.0,It is slightly more difficult than Excel or Wo...,4.0,2400.0,8192.0,2019-02-27 20:30:18,-100.0,NaN
12634,2019-03-02 16:05:07,3,NaN,NaN,NaN,NaN,-100,NaN,3.0,NaN,8.0,1992.0,16384.0,2019-03-02 08:05:07,-100.0,NaN
12890,2019-03-03 15:45:09,5,NaN,NaN,NaN,NaN,100,NaN,5.0,NaN,4.0,2494.0,8192.0,2019-03-03 07:45:09,100.0,NaN
12892,2019-03-03 15:59:08,3,It would be interesting to integrate Office wi...,NaN,NaN,NaN,-100,NaN,3.0,It would be interesting to integrate Office wi...,4.0,2808.0,16384.0,2019-03-03 07:59:08,-100.0,NaN


In [68]:
test = df_SL[df_SL.ProcSpeedMHz_other.notnull()].ProcSpeedMHz_Kusto != df_SL[df_SL.ProcSpeedMHz_other.notnull()].ProcSpeedMHz_other

print(df_SL[df_SL.ProcSpeedMHz_other.notnull()].loc[test,common_fields].shape)
df_SL[df_SL.ProcSpeedMHz_other.notnull()].loc[test,common_fields].head()

(26, 16)


,DateTime_Kusto,Rating_Kusto,Verbatim_Kusto,ProcessorCount_Kusto,ProcSpeedMHz_Kusto,RamMB_Kusto,ratingvalue_Kusto,SysVolUsagePercent_Kusto,Rating_other,Verbatim_other,ProcessorCount_other,ProcSpeedMHz_other,RamMB_other,DateTime_other,ratingvalue_other,SysVolUsagePercent_other
9629,2019-02-22 14:20:36,4,NaN,NaN,NaN,NaN,0,NaN,4.0,NaN,4.0,1992.0,4096.0,2019-02-22 06:20:36,0.0,NaN
11719,2019-02-28 04:30:18,3,It is slightly more difficult than Excel or Wo...,NaN,NaN,NaN,-100,NaN,3.0,It is slightly more difficult than Excel or Wo...,4.0,2400.0,8192.0,2019-02-27 20:30:18,-100.0,NaN
12634,2019-03-02 16:05:07,3,NaN,NaN,NaN,NaN,-100,NaN,3.0,NaN,8.0,1992.0,16384.0,2019-03-02 08:05:07,-100.0,NaN
12890,2019-03-03 15:45:09,5,NaN,NaN,NaN,NaN,100,NaN,5.0,NaN,4.0,2494.0,8192.0,2019-03-03 07:45:09,100.0,NaN
12892,2019-03-03 15:59:08,3,It would be interesting to integrate Office wi...,NaN,NaN,NaN,-100,NaN,3.0,It would be interesting to integrate Office wi...,4.0,2808.0,16384.0,2019-03-03 07:59:08,-100.0,NaN


**Decision: Use the fields from our data (DataMart)**

In [69]:
df_SL.drop(list(df_SL.columns[df_SL.columns.str.endswith('_other')]),axis=1,inplace=True)
df_AL.drop(list(df_AL.columns[df_AL.columns.str.endswith('_other')]),axis=1,inplace=True)

In [70]:
df_SL.iloc[:,df_SL.columns.str.startswith('ProcSpeedMHz')].head(3) #now only one ProcSpeedMHz column

,ProcSpeedMHz_Kusto
0,3193.0
1,2594.0
2,3493.0


# EDA

In [71]:
df_SL.loc[:,df_SL.dtypes!=object].Sessions_7.notnull()

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
83373    False
83374    False
83375    False
83376    False
83377    False
83378    False
83379    False
83380    False
83381    False
83382    False
83383    False
83384    False
83385    False
83386    False
83387    False
83388    False
83389    False
83390    False
83391    False
83392    False
83393    False
83394    False
83395    False
83396    False
83397    False
83398    False
83399    False
83400    False
83401    False
83402    False
Name: Sessions_7, Length: 83403, dtype: bool

In [72]:
def make_scatter_matrix(X,figsize_dim):
    scatter_matrix = pd.plotting.scatter_matrix(X,figsize=figsize_dim,
                              diagonal='kde',alpha=0.2)

    [s.xaxis.label.set_rotation(300) for s in scatter_matrix.reshape(-1)]
    [s.yaxis.label.set_rotation(60) for s in scatter_matrix.reshape(-1)]

    #May need to offset label when rotating to prevent overlap of figure
    [s.get_yaxis().set_label_coords(-0.3,0.5) for s in scatter_matrix.reshape(-1)]

    [plt.setp(item.yaxis.get_label(), 'size', 20) for item in scatter_matrix.ravel()]
    #x labels
    [plt.setp(item.xaxis.get_label(), 'size', 20) for item in scatter_matrix.ravel()]

    #Hide all ticks
    # [s.set_xticks(()) for s in sm.reshape(-1)]
    # [s.set_yticks(()) for s in sm.reshape(-1)]

In [73]:
def outlier_detect(df, col_name):
    IsOutlier = df[col_name] >= df[col_name].quantile(q=0.75)+1.5*(df[col_name].quantile(q=0.75)-df[col_name].quantile(q=0.25))
    print(col_name, ' has ',sum(IsOutlier),' outliers')
    df = df[~IsOutlier]
    return df

In [74]:
df_SL.Sessions_28.value_counts()

0.0    41595
Name: Sessions_28, dtype: int64

In [75]:
def feature_engineer(df):
    df['SysVolFreeSpaceGB'] = round(df['Device_SysVolFreeSpaceMB']/1000,0)
    df['SysVolSizeGB'] = round(df['Device_SysVolSizeMB']/1000,0)
    df['RamGB'] = round(df['RamMB_Kusto']/1048,0)
    df['ProcSpeedGHz'] = round(df['ProcSpeedMHz_Kusto']/1000,0)
    df['SysVolUsagePercent'] = 1-(df['SysVolFreeSpaceGB']/df['SysVolSizeGB'])
    df = df[df['Device_SysVolFreeSpaceMB']<=2000000]
    df = df[df['Device_SysVolSizeMB']<=2000000]
    df = df[df['SysVolUsagePercent']>=0]
    #df = df[df['IsOutlier?']==False] #drop duration outliers for now
    df = df[df.RamGB<=32] #drop RAM outliers for now
    df = df[df.ProcessorCount_Kusto<=16] #drop ProcCount outliers for now
    df = df[df.ProcSpeedGHz<=4] #Drop ProcSpeed outliers for now
    return df

df_SL = feature_engineer(df_SL)
df_AL = feature_engineer(df_AL)

In [ ]:
make_scatter_matrix(df_SL.iloc[:,(df_SL.dtypes=='float64').values],(15,15))

In [17]:
df_SL.columns[df_SL.columns.str.endswith('usto')]


Index(['DateTime_Kusto', 'Rating_Kusto', 'Verbatim_Kusto',
       'ProcessorCount_Kusto', 'ProcSpeedMHz_Kusto', 'RamMB_Kusto',
       'ratingvalue_Kusto', 'SysVolUsagePercent_Kusto'],
      dtype='object')

In [18]:
df_SL.columns[df_SL.columns.str.startswith('Device')]


Index(['DeviceManufacturer', 'DeviceModel', 'Device_SysVolSizeMB',
       'Device_SysVolFreeSpaceMB', 'DeviceId'],
      dtype='object')

In [20]:
#sum(df_SL[''] != df_SL['Feedback_RatingValue']),sum(df_AL[''] != df_AL['Feedback_RatingValue'])

In [21]:
print(df_SL.shape, df_AL.shape)
df_SL = df_SL.loc[:,~df_SL.columns.duplicated()]
df_AL = df_AL.loc[:,~df_AL.columns.duplicated()]
print(df_SL.shape, df_AL.shape)

(41292, 361) (49234, 361)
(41292, 361) (49234, 361)


In [76]:
df_SL = df_SL.iloc[:,~df_SL.columns.str.contains('Id')]
df_SL = df_SL.iloc[:,~df_SL.columns.str.contains('ID')]
df_SL = df_SL.iloc[:,~df_SL.columns.str.contains('Sku')]
df_SL = df_SL.iloc[:,~df_SL.columns.str.contains('Verbatim')]
df_SL = df_SL.iloc[:,~df_SL.columns.str.contains('Time')]

df_AL = df_AL.iloc[:,~df_AL.columns.str.contains('Id')]
df_AL = df_AL.iloc[:,~df_AL.columns.str.contains('ID')]
df_AL = df_AL.iloc[:,~df_AL.columns.str.contains('Sku')]
df_AL = df_AL.iloc[:,~df_AL.columns.str.contains('Verbatim')]
df_AL = df_AL.iloc[:,~df_AL.columns.str.contains('Time')]

In [77]:
((df_SL.dtypes=='int64') | (df_SL.dtypes=='float64')).values

array([False,  True, False, False,  True, False,  True,  True, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False, False, False, False,
       False,  True,  True,  True, False, False,  True, False, False,
       False, False, False,  True,  True, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
        True,  True,  True,  True,  True, False, False, False, False,
        True,  True,  True, False, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False, False, False, False, False,
       False,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,

In [78]:
df_SL.iloc[:,((df_SL.dtypes=='int64') | (df_SL.dtypes=='float64')).values].head(4)

,Source.1,Rating_Kusto,RatingValue,RatingScale,IsValidFlight,ProcessorCount_Kusto,ProcSpeedMHz_Kusto,RamMB_Kusto,NumProcPhysCores,SysVolSizeMB,...,Device_SysVolSizeMB,Device_SysVolFreeSpaceMB,rn,DurationSeconds,Sessions_7,Sessions_14,Sessions_28,SysVolFreeSpaceGB,SysVolSizeGB,SysVolUsagePercent
131,NaN,3,-100,5,1,8.0,2304.0,8192.0,4.0,613882.0,...,613882.0,527256.0,1.0,5.793433,0.0,0.0,0.0,527.0,614.0,0.141694
144,NaN,5,100,5,1,4.0,2712.0,4096.0,2.0,230197.0,...,230197.0,170322.0,1.0,1.631547,0.0,0.0,0.0,170.0,230.0,0.260870
197,NaN,3,-100,5,1,4.0,2395.0,4096.0,2.0,459029.0,...,459029.0,356620.0,1.0,2.061298,0.0,0.0,0.0,357.0,459.0,0.222222
292,NaN,4,0,5,1,4.0,2666.0,8192.0,4.0,476387.0,...,476387.0,414202.0,1.0,1.734153,0.0,0.0,0.0,414.0,476.0,0.130252


In [164]:
X_SL = df_SL.loc[:,['ProcessorArchitecture','PrimaryDiskType',
              'Scenario','Feedback_SurveyName',
              'Feedback_UserType','Feedback_App','OsVersionName',
              'RamGB','ProcessorCount_Kusto','ProcSpeedGHz','DurationSeconds',
              'SysVolFreeSpaceGB','SysVolSizeGB','SysVolUsagePercent']]
              #'AppArchitecture', 'Country', 'AppVersion','Sessions_28']]


In [165]:
X_AL = df_AL.loc[:,['ProcessorArchitecture','PrimaryDiskType',
              'Scenario','Feedback_SurveyName',
              'Feedback_UserType','Feedback_App','OsVersionName',
              'RamGB','ProcessorCount_Kusto','ProcSpeedGHz','DurationSeconds',
              'SysVolFreeSpaceGB','SysVolSizeGB','SysVolUsagePercent']]
              #'AppArchitecture', 'Country', 'AppVersion','Sessions_28']]


In [166]:
def get_dummies(X):
    StrTo_Cat = X.columns[X.dtypes=='object']
    print(StrTo_Cat)
    for x in StrTo_Cat:
        if (X[x].nunique() >1)&(X[x].nunique() <=5):
            print(x)
            X.loc[X[x].isnull(), x] ='No_Data'
            X[x]= X[x].astype('category')
            X = pd.concat([X, pd.get_dummies(X[x], prefix=x, prefix_sep='_',)], axis = 1)
        del X[x]
        #pd.data[StrTo_Cat]= data[StrTo_Cat].astype('category')
    return X
    
X_SL = get_dummies(X_SL)
X_SL = X_SL.iloc[:,~X_SL.columns.str.endswith('No_Data')]
                  

Index(['ProcessorArchitecture', 'PrimaryDiskType', 'Scenario',
       'Feedback_SurveyName', 'Feedback_UserType', 'Feedback_App',
       'OsVersionName'],
      dtype='object')
ProcessorArchitecture
PrimaryDiskType
Scenario
Feedback_UserType
Feedback_App


In [167]:
X_AL = get_dummies(X_AL)
X_AL = X_AL.iloc[:,~X_AL.columns.str.endswith('No_Data')]

Index(['ProcessorArchitecture', 'PrimaryDiskType', 'Scenario',
       'Feedback_SurveyName', 'Feedback_UserType', 'Feedback_App',
       'OsVersionName'],
      dtype='object')
ProcessorArchitecture
PrimaryDiskType
Scenario
Feedback_UserType
Feedback_App


In [124]:
X_SL.head().to_csv('test_matrix.csv')

PermissionError: [Errno 13] Permission denied: 'test_matrix.csv'

In [168]:
X = {'Suite':X_SL,
     'App':X_AL}
y = {'Suite':df_SL['Feedback_RatingValue'].replace([100,0],1).replace(-100,0),
     'App':df_AL['Feedback_RatingValue'].replace([100,0],1).replace(-100,0)}

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X['Suite'], y['Suite'], test_size=0.1)

In [100]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((37161, 22), (4130, 22), (37161,), (4130,))

In [52]:
X_train.head().to_csv('test_matrix.csv')

In [126]:
### ONLY RUN IF THERE IS NO OUTPUT ###
rf = RandomForestClassifier(random_state=0)
param_grid = { 
    'n_estimators': [50, 100, 250, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [2,5,10,None],
    'oob_score': [True,False]}

CV_rfc = GridSearchCV(estimator=rf, n_jobs=-1, param_grid=param_grid, verbose=10, scoring='neg_log_loss',cv= 5)
CV_rfc.fit(X_train, y_train)
print(CV_rfc.best_params_, CV_rfc.best_score_)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   25.3s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   37.1s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  5

{'max_depth': 5, 'max_features': 'auto', 'n_estimators': 500, 'oob_score': True} -0.4545966882293592


In [44]:
CV_rfc.best_params_

{'max_depth': 5,
 'max_features': 'auto',
 'n_estimators': 500,
 'oob_score': True}

In [128]:
print(RandomForestClassifier(random_state=0).fit(X_train,
                                           y_train).score(X_test,y_test))


rfc = RandomForestClassifier(random_state = 0, **CV_rfc.best_params_)
rfc.fit(X_train,y_train)
print(rfc.score(X_test,y_test))

learners_S = rfc.feature_importances_.argsort()[::-1]

features_S = pd.DataFrame(X_train.columns[learners_S], rfc.feature_importances_[learners_S])
features_S = features[features.index>0.025]
features_S

C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7830508474576271
0.8249394673123487


,0
0.140480,DurationSeconds
0.121764,SysVolSizeGB
0.115869,Feedback_App_Word
0.108700,SysVolFreeSpaceGB
0.103993,SysVolUsagePercent
0.058313,Feedback_UserType_Consumer
0.057148,Feedback_App_Excel
0.055590,Feedback_UserType_Commercial
0.046602,RamGB
0.038810,ProcSpeedGHz


In [111]:
def singular_test(matrix):
    if np.linalg.cond(X['Suite']) < 1/sys.float_info.epsilon:
        i = np.linalg.inv(x)
    else:
        #handle it
        i = 'yes'
    return i
    
singular_test(X['Suite'])

'yes'

In [129]:
X['Suite'].iloc[:,:1].head(5)

,RamGB
131,8.0
144,4.0
197,4.0
292,8.0
297,8.0


In [190]:
mlogit = sm.Logit(y['Suite'], X['Suite'])
fmlogit = mlogit.fit()

print(fmlogit.summary())

Optimization terminated successfully.
         Current function value: 0.455225
         Iterations 6
                            Logit Regression Results                            
Dep. Variable:     Feedback_RatingValue   No. Observations:                41291
Model:                            Logit   Df Residuals:                    41274
Method:                             MLE   Df Model:                           16
Date:                  Thu, 14 Nov 2019   Pseudo R-squ.:                0.005838
Time:                          15:03:29   Log-Likelihood:                -18797.
converged:                         True   LL-Null:                       -18907.
                                          LLR p-value:                 4.917e-38
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
RamGB                            0.0012      0.003      

C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:1092: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))
C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [191]:
SuiteOutput = fmlogit.summary2().tables[1]
SuiteStatSig = SuiteOutput[SuiteOutput['P>|z|']<0.1]
SuiteStatSig

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
SysVolFreeSpaceGB,0.000548,0.000170,3.219318,0.001285,0.000214,0.000881
SysVolSizeGB,-0.000356,0.000130,-2.747408,0.006007,-0.000611,-0.000102
SysVolUsagePercent,0.298638,0.085463,3.494378,0.000475,0.131135,0.466142
PrimaryDiskType_HDD,0.075960,0.044422,1.709979,0.087270,-0.011105,0.163025
PrimaryDiskType_SSD,-0.115263,0.030613,-3.765233,0.000166,-0.175263,-0.055264


In [131]:
X_train, X_test, y_train, y_test = train_test_split(X['App'], y['App'], test_size=0.1)
### ONLY RUN IF THERE IS NO OUTPUT ###
rf = RandomForestClassifier(random_state=0)
param_grid = { 
    'n_estimators': [50, 100, 250, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [2,5,10,None],
    'oob_score': [True,False]}

CV_rfc = GridSearchCV(estimator=rf, n_jobs=-1, param_grid=param_grid, verbose=10, scoring='neg_log_loss',cv= 5)
CV_rfc.fit(X_train, y_train)
print(CV_rfc.best_params_, CV_rfc.best_score_)

print(RandomForestClassifier(random_state=0).fit(X_train,
                                           y_train).score(X_test,y_test))


rfc = RandomForestClassifier(random_state = 0, **CV_rfc.best_params_)
rfc.fit(X_train,y_train)
print(rfc.score(X_test,y_test))

learners_A = rfc.feature_importances_.argsort()[::-1]

features_A = pd.DataFrame(X_train.columns[learners_A], rfc.feature_importances_[learners_A])
features_A = features_A[features_A.index>0.025]
features_A

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   17.3s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   42.1s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   55.6s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  4

{'max_depth': 5, 'max_features': 'auto', 'n_estimators': 500, 'oob_score': True} -0.3807221737496463


C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.847684809098294
0.8759138911454102


,0
0.143439,DurationSeconds
0.121789,Feedback_App_Word
0.117849,SysVolFreeSpaceGB
0.107406,SysVolUsagePercent
0.106141,SysVolSizeGB
0.064225,Feedback_UserType_Commercial
0.061885,Feedback_UserType_Consumer
0.061225,Feedback_App_Excel
0.042563,RamGB
0.039347,ProcSpeedGHz


In [176]:
mlogit = sm.Logit(y['App'], X['App'])
fmlogit = mlogit.fit()

print(fmlogit.summary())

Optimization terminated successfully.
         Current function value: 0.379858
         Iterations 7
                            Logit Regression Results                            
Dep. Variable:     Feedback_RatingValue   No. Observations:                49234
Model:                            Logit   Df Residuals:                    49217
Method:                             MLE   Df Model:                           16
Date:                  Thu, 14 Nov 2019   Pseudo R-squ.:                0.005566
Time:                          14:59:44   Log-Likelihood:                -18702.
converged:                         True   LL-Null:                       -18807.
                                          LLR p-value:                 1.015e-35
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
RamGB                           -0.0113      0.003     -

In [189]:
AppOutput = fmlogit.summary2().tables[1]
AppStatSig = AppOutput[AppOutput['P>|z|']<0.1]
AppStatSig

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
RamGB,-0.011293,0.003068,-3.681485,0.000232,-0.017306,-0.005281
DurationSeconds,-0.000839,0.000485,-1.730838,0.083481,-0.001789,0.000111
SysVolUsagePercent,0.159070,0.087878,1.810110,0.070279,-0.013169,0.331308
PrimaryDiskType_HDD,0.101176,0.044831,2.256826,0.024019,0.013309,0.189044
